In [ ]:
import nltk
import re
import heapq
import requests
import cv2
import numpy as np
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
URL = "http://192.168.1.4:8080/shot.jpg"
ker = np.ones((1,1), np.uint8)

def summaryfunc(paragraph):
    
    cleanedpara = re.sub(r'\[[0-9]*\]',' ', paragraph)
    cleanedpara = re.sub(r'\s+',' ', cleanedpara)

    sen_tokens = nltk.sent_tokenize(cleanedpara)

    cleanedpara = re.sub(r'[^a-zA-Z]',' ', cleanedpara)
    cleanedpara = re.sub(r'\s+',' ', cleanedpara)


    word_token = nltk.word_tokenize(cleanedpara)

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequency = {}

    for word in word_token:
        if word not in stopwords:
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else:
                word_frequency[word] += 1

    maximum_frequency_word = max(word_frequency.values())

    for word in word_frequency.keys():
        word_frequency[word] = (word_frequency[word] / maximum_frequency_word)

    sentence_score = {}

    for sentence in sen_tokens:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_frequency.keys():
                if (len(sentence.split(' '))) < 30:
                    if sentence not in sentence_score.keys():
                        sentence_score[sentence] = word_frequency[word]
                    else:
                        sentence_score[sentence] += word_frequency[word]

    summary = heapq.nlargest(4, sentence_score, key = sentence_score.get)
    print(summary)

def display_img(pic):
    cv2.imshow("pic", pic)
    cv2.waitKey(0)

def get_text(frr):
    text = pytesseract.image_to_string(frr, lang='eng')
    print("SUMMARY\n\n")
    summaryfunc(text)
    print("\n\n\noriginal paragraph")
    print(text)
    display_img(frr)

def process_img(im):
    im = cv2.resize(im, (700, 700))
    #ret, im = cv2.threshold(img, 125, 255, cv2.THRESH_BINARY)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.dilate(im, ker, iterations=1)
    im = cv2.erode(im, ker, iterations=1)
    im = cv2.resize(im, (700, 700))
    #display_img(im)
    get_text(im)

#v1 = cv2.VideoCapture(0)
#path = " your path "
#img = cv2.imread(path)

while True:
    #ret, frame = v1.read()
    img_res = requests.get(URL)
    img_arr = np.array(bytearray(img_res.content), np.uint8)
    img = cv2.imdecode(img_arr, -1)

    cv2.imshow('IPWebcam', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow('IPWebcam')
        break
#img = cv2.imread('Screenshot.png')

process_img(img)
cv2.destroyAllWindows()